This notebook demonstrates how to use the multilayer_simulator package to set up, run, and visualise a 1D optical simulation in combination with the Lumerical STACK solver and Python API.

We begin by importing the relevant modules:

In [ ]:
import lumapi
import numpy as np
from multilayer_simulator import Layer, Multilayer, Simulation
from multilayer_simulator.lumerical_classes import LumericalMaterial, LumericalOscillator, LumericalSTACK, format_STACK
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts

hv.extension('bokeh')

Start the Lumerical session as directed in the [lumapi documentation](https://optics.ansys.com/hc/en-us/articles/360041873053):

In [ ]:
fdtd = lumapi.FDTD()

The `simulation` object handles the running of the simulation, including run-time parameters such as frequency and angle. It needs to be provided with a `Structure` to simulate and an `Engine` to perform the simulation.

The simplest `Structure` is a `Layer`. It *can* be initialized by providing an appropriate index function and thickness as arguments:

In [ ]:
def index_of_vacuum(frequencies = None, component = 1):
    if frequencies is not None:
        frequencies = np.atleast_1d(frequencies)
        return np.full_like(frequencies, 1)
    else:
        return np.array([1])

In [ ]:
halfspace = Layer(index_of_vacuum) # default thickness is 0

However, it is recommended to instead define a `Material` which has an `index` method and use the `Layer.from_material()` constructor.

We can access or define a material in the Lumerical database using the [Lumerical scripting language](https://optics.ansys.com/hc/en-us/articles/360037228834) with `lumapi`. The `LumericalMaterial` class packages the relevant script commands for us and allows us to interact with the material in the database like a `Material` object.

In [ ]:
glass = LumericalMaterial(session=fdtd, name="SiO2 (Glass) - Palik")

glass.get_property("all")

Now we can define a 1 micron thick layer of glass:

In [ ]:
glass_layer = Layer.from_material(glass, thickness=1e-6)

As well as accessing existing materials in the database, we can define new ones. Here we use the `LumericalOscillator` subclass of `LumericalMaterial` to easily add a Lorentz oscillator to the material database, and construct another layer from it:

In [ ]:
try: # In case the material has already been defined in the database, remove it to avoid raising an error
    oscillator
except NameError:
    pass
else:
    oscillator.delete()
oscillator = LumericalOscillator(fdtd, name='Example Oscillator', permittivity=1, lorentz_resonance=6E14, lorentz_permittivity=1, lorentz_linewidth=1E13)

excitonic_layer = Layer.from_material(oscillator, 5e-6)

The minimal structure to coherently define an optical model is a three-layer `Multilayer`. The first and last layer define the material of the incidence and exit semi-infinite half-spaces, and usually have zero thickness. If they have a positive thickness, layers of that thickness are added to the start and end of the model (or equivalently, the transmission and reflection monitors are moved that distance out from the edges). This could be relevant for absorbing media.

We can define a `Multilayer` from a list of `Layer`s.

In [ ]:
multilayer = Multilayer([halfspace, glass_layer, excitonic_layer, halfspace])

Having defined the structure, we now need the `Engine`. The `LumericalSTACK` engine packages together the `stackrt` and `stackfield` Lumerical script commands.

In [ ]:
engine = LumericalSTACK(fdtd)

That's it! Now we can define and run the simulation:

In [ ]:
frequencies = np.linspace(4e14, 8e14, 100)
angles = np.arange(0, 41, 10)
simulation = Simulation(multilayer, engine, frequencies=frequencies, angles=angles)
simulation.simulate()

`Simulation.simulate()` can take parameters which override the attributes it was constructed with, for convenience:

In [ ]:
low_frequency_zero_incidence_simulation = simulation.simulate(frequencies=3e14, angles=0, keep_data=False)

This data can already be visualised using your preferred plotting library:

In [ ]:
wavelengths = simulation.data[0]['lambda']
reflectance_s = simulation.data[0]['Rs'][:,0]

plt.plot(wavelengths, reflectance_s)

Alternatively, a `DataFormatter` can be applied to it. My preferred format is xarray, but you can write your own formatter for pandas or even dictionaries.

The `DataFormatter` can not only parse the data into a particular format, it can also perform predictable data munging such as calculating absorptance or field magnitude.

In [ ]:
xarray_datasets = format_STACK.from_tuple(simulation.data).to_xarray_dataset(stackrt_args={'variables': ['Rs', 'Rp', 'Ts', 'Tp']})

In [ ]:
xarray_datasets[0]

In [ ]:
xarray_datasets[1]

Formatted data can then be explored using the standard visualisation libraries. A future version of this package may include some helpful `Visualiser` classes to handle common tasks.

In [ ]:
field_data = xarray_datasets[1]
field_data['|Es|^2'].sel(theta=0, frequency=6e14, method='nearest').plot(x='z')
# field_data['Es'].squeeze().sel(theta=0, frequency=6e14, method='nearest').sel(vector='j').real.plot.line(x='z')

Here is a recipe for using the defined classes to dynamically explore a parameter space using Holoviews:

In [ ]:
def data_from_params(theta, glass_thickness_microns, oscillator_resonance_terahertz):
    glass_layer.thickness = glass_thickness_microns * 1e-6
    oscillator.lorentz_resonance = oscillator_resonance_terahertz * 1e12

    data = simulation.simulate(angles=theta, keep_data=False)
    rt_data, field_data = format_STACK.from_tuple(data).to_xarray_dataset(
        stackrt_args={"variables": ["Rs", "Rp", "Ts", "Tp"]},
        stackfield_args={"variables": ["Es", "Ep"]},
    )

    return rt_data, field_data

In [ ]:
def graphs_from_data(rt_data, field_data, frequency_terahertz):
    frequency = frequency_terahertz * 1e12
    rt_curve_s = hv.Overlay([hv.Curve((rt_data['wavelength'], rt_data[var].squeeze()), 'Wavelength', 'Intensity', label=var) for var in ['Rs', 'Ts', 'As']])
    rt_curve_p = hv.Overlay([hv.Curve((rt_data['wavelength'], rt_data[var].squeeze()), 'Wavelength', 'Intensity', label=var) for var in ['Rp', 'Tp', 'Ap']])
    field_curves = hv.Layout([hv.Curve((field_data['z'], field_data.squeeze().sel(frequency=frequency, method='nearest')[var]), 'z', 'Intensity', label=var) for var in ['|Es|^2', '|Ep|^2']])
    layout = rt_curve_s + field_curves[0] + rt_curve_p + field_curves[1]
    return layout.cols(2)

In [ ]:
def graphs_from_params(frequency_terahertz, theta, glass_thickness_microns, oscillator_resonance_terahertz):
    data = data_from_params(theta, glass_thickness_microns, oscillator_resonance_terahertz)
    graphs = graphs_from_data(*data, frequency_terahertz)
    return graphs

In [ ]:
parameter_ranges = {
    'Frequency (Terahertz)': (100, 1000),
    'Angle (Degrees)': (0, 80),
    'Glass Thickness (Microns)': (0.1, 10),
    'Oscillator Resonance Frequency (Terahertz)': (100, 1000)
}
parameters = list(parameter_ranges.keys())

hv.DynamicMap(graphs_from_params, kdims=parameters).redim.range(**parameter_ranges).opts(opts.Curve(width=500))